<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/_gp_bart_recurrent_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* kobart 공식 깃헙

https://github.com/SKT-AI/KoBART#examples

https://github.com/seujung/KoBART-summarization

1. kobart 불러오기

2. kobart fine-tuning에 맞게 훈련 데이터 형태 변환 (.tsv) (gp_bart_data_processing.ipynb에서 수행)

3. kobart fine-tuning 

4. train된 kobart에 train set의 일기 데이터셋을 넣어 공감코멘트 생성 (.csv)

5. (4)의 output으로 기존 kobart 모델 추가 train (데이터 되먹임)

6. (4) ~ (5) 과정을 반복 (생성되는 공감코멘트가 어느정도의 성능으로 수렴할 때까지)

### 1. kobart 불러오기 ~ 3. kobart fine-tuning

In [ ]:
# KoBART-summarization git 다운받고 -> 
# KoBART-summarization 내부의 data 디렉토리에 train, test.tsv data 파일(data/bart_train_data.tsv) 내 걸로 바꾸고 -> 
# KOBART의 requirment와 binary 인스톨 및 다운로드 하고 ->
# KoBART의 python train.py 실행시켜 fine-tuning 수행: 
# !python train.py  --gradient_clip_val 1.0  \
#                  --max_epochs 50 \
#                  --default_root_dir logs \
#                  --gpus 1 \
#                  --batch_size 4 \
#                  --num_workers 4

# train 후 생성되는 공감코멘트 quailty 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [ ]:
# !git clone https://github.com/seujung/KoBART-summarization.git

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization'

/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x6482e000 @  0x7f56f8d91615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 18 kB/s 
     |████████████████████████████████| 2.5 MB 78.7 MB/s 
     |████████████████████████████████| 813 kB 76.8 MB/s 
     |████████████████████████████████| 8.3 MB 48.4 MB/s 
     |████████████████████████████████| 880 kB 63.5 MB/s 
     |████████████████████████████████| 3.3 MB 54.8 MB/s 
     |████████████████████████████████| 829 kB 77.8 MB/s 
     |████████████████████████████████| 636 kB 66.9 MB/s 
     |█████████████████████

In [ ]:
!pip install torchmetrics==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 329 kB 14.0 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.9.3
    Uninstalling torchmetrics-0.9.3:
      Successfully uninstalled torchmetrics-0.9.3


In [ ]:
# OSError
# https://momozzing.github.io/error%20resolution/torchtext-OSError/
# https://github.com/pytorch/text/issues/1342#issuecomment-993928516
# UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: 
# https://programmerah.com/solved-torchvision-error-userwarning-failed-to-load-image-python-extension-could-not-find-module-48667/
!pip install torchtext==0.11.0
!pip install torchvision==0.11.2
!pip install torchaudio===0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.0 MB 11.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x3a196000 @  0x7f57b5392615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing in

In [ ]:
# AttributeError: 'Trainer' object has no attribute '_data_connector'
# https://zorba-blog.tistory.com/23
!pip install pytorch_lightning==1.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 15.5 MB/s 
  Attempting uninstall: pyDeprecate
    Found existing installation: pyDeprecate 0.3.0
    Uninstalling pyDeprecate-0.3.0:
      Successfully uninstalled pyDeprecate-0.3.0
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.3.8
    Uninstalling pytorch-lightning-1.3.8:
      Successfully uninstalled pytorch-lightning-1.3.8


In [ ]:
import sys
sys.version
# pyyhon 3.7.x version으로 맞출 것

'3.7.14 (default, Sep  8 2022, 00:06:44) \n[GCC 7.5.0]'

In [ ]:
# 기존 max_epochs=50
# train, test data를 내 data로 수정하여 실행하기
!python train.py  --gradient_clip_val 1.0  \
                 --max_epochs 10 \
                 --default_root_dir logs \
                 --gpus 1 \
                 --batch_size 4 \
                 --num_workers 4 \
                 --model_path '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/logs/model_chp/epoch=02-val_loss=3.203.ckpt'

Downloading: 100% 4.00/4.00 [00:00<00:00, 4.33kB/s]
Downloading: 100% 111/111 [00:00<00:00, 111kB/s]
Downloading: 100% 682k/682k [00:00<00:00, 1.98MB/s]
INFO:root:Namespace(accelerator=None, accumulate_grad_batches=None, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=None, checkpoint_path=None, default_root_dir='logs', detect_anomaly=False, deterministic=False, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, flush_logs_every_n_steps=None, gpus=1, gradient_clip_algorithm=None, gradient_clip_val=1.0, ipus=None, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=10, max_len=512, max_steps=-1, max_time=None, min_epochs=None, min_steps=None, model_path='/content/d

#### test + 4. train된 kobart에 train set의 일기 데이터셋을 넣어 공감코멘트 생성 (.csv)

In [ ]:
import numpy as np
import pandas as pd
import csv

def drop_na(df):
  df= df.dropna()
  df= df.reset_index(drop=True)
  return df

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

In [ ]:
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization') # fine-tuning된 모델로

In [ ]:
# hparam_path= '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/logs/tb_logs/default/version_0/hparams.yaml'
# model_binary_path= '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/logs/model_chp/epoch=00-val_loss=2.590.ckpt'

In [ ]:
# !python get_model_binary.py --hparams '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/logs/tb_logs/default/version_0/hparams.yaml' \
#                             --model_binary '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/logs/model_chp/epoch=00-val_loss=2.590.ckpt'

In [ ]:
# !streamlit run infer.py

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/graduation_project

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [ ]:
!pip install transformers==4.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pytorch_lightning as pl

In [ ]:
class BartModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')  
      
    def forward(self):
        pass
 
# 불러올 모델 파일 바꾸기
ckpt = torch.load('/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/logs/model_chp/epoch=02-val_loss=3.203.ckpt')
   
bart_model = BartModel()
bart_model.load_state_dict(ckpt['state_dict'])
bart_model.model.save_pretrained("./KoBART-summarization/logs/model_chp")

In [ ]:
model = BartForConditionalGeneration.from_pretrained('./KoBART-summarization/logs/model_chp')

- test

In [ ]:
test_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/data/test.tsv', sep='\t')
test_data= test_data[['news']] # test_text

test_data= drop_na(test_data)

In [ ]:
text_list = test_data['news']
summary_list=list()

In [ ]:
text_list = test_data['news']
summary_list=list()

for i, text in enumerate(text_list):
  if len(text) + len(" TL;DR ") > 800:
    text = text[0:400] + text[len(text)-400:len(text)]
  text= text + " TL;DR "

  text = text.replace('\n', ' ')

  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
  output= tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  summary_list.append(output)
  if i%10 == 0:
    print('test {} 개 진행중'.format(i))

test 0 개 진행중
test 10 개 진행중
test 20 개 진행중
test 30 개 진행중


In [ ]:
summary_list

['나를 사랑하는하는 연습 어떻게 하면 나를 사랑할 수 있을까요? 자책하는 습관부터 버려야 돼요. 자책하는 습관부터 버려야 돼요. 자책은 사실 나를 갉아 먹는 일이거든, 자책하는 습관부터 버려야 돼요.',
 '일본어를 어디서 배웠는지, 뭐 하려고 이 호텔에 왔는지 그런 질문에 답을 해줘야 했군요. 정말 화가 났을 것 같아요. 그래도 김동률 콘서트는 정말 멋진 공연이었다니 다행이에요.',
 '수간호사선선생님의 ot를 간단히 들으셨군요. 바이탈사인을 척정하고 이브 루틴 바이탈우지보다 안좋은 상태인 것 같아요. 빨리 상황이 나아졌으면 좋겠어요.',
 '반포한강에 다녀오셨군요! 한강에서 남준이를 마주치는 상상도 해보고, 야경이 아름다웠으면 좋겠네요.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ',
 '아버지와 둘이 햄버거를 먹으셨군요. 저도 햄버거를 먹었으면 좋겠네요.',
 '오늘 밤 미치려고 작정한 것 같아요. 욕을 안하고 글을 쓸 수가 없어서 부끄럽고요. 그래도 좋은 사람이 생기길 바랄게요.',
 '강아지와 고양이를 미용하는 애견미용사(펫 스타일리스트)이시군요. 분

In [ ]:
test_data['summary'] = summary_list

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>> 훈련 반복할 때마다 파일명 바꾸기 default -> 1 -> 2...
test_data.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/kobart-kogpt_output/kobart_test_2.csv') 

test 결과 보고 -> train된 kobart에 train set의 일기 데이터셋을 넣어 공감코멘트 생성 (.csv)

In [ ]:
next_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/data/train.tsv', sep='\t')
next_data= next_data[['news']]

next_data= drop_na(next_data)

In [ ]:
text_list = next_data['news']
summary_list=list()

In [ ]:
text_list = next_data['news']
summary_list=list()

for i, text in enumerate(text_list):
  if len(text) + len(" TL;DR ") > 800:
    text = text[0:400] + text[len(text)-400:len(text)]
  text= text + " TL;DR "

  text = text.replace('\n', ' ')

  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
  output= tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  summary_list.append(output)
  if i%10 == 0:
    print('{} 개 진행중'.format(i))

0 개 진행중
10 개 진행중
20 개 진행중
30 개 진행중
40 개 진행중
50 개 진행중
60 개 진행중
70 개 진행중
80 개 진행중
90 개 진행중
100 개 진행중
110 개 진행중
120 개 진행중
130 개 진행중
140 개 진행중
150 개 진행중
160 개 진행중
170 개 진행중
180 개 진행중
190 개 진행중
200 개 진행중
210 개 진행중
220 개 진행중
230 개 진행중
240 개 진행중
250 개 진행중
260 개 진행중
270 개 진행중
280 개 진행중
290 개 진행중
300 개 진행중
310 개 진행중
320 개 진행중
330 개 진행중
340 개 진행중
350 개 진행중
360 개 진행중
370 개 진행중
380 개 진행중
390 개 진행중
400 개 진행중
410 개 진행중
420 개 진행중
430 개 진행중
440 개 진행중
450 개 진행중
460 개 진행중
470 개 진행중
480 개 진행중
490 개 진행중
500 개 진행중
510 개 진행중
520 개 진행중
530 개 진행중
540 개 진행중
550 개 진행중
560 개 진행중
570 개 진행중
580 개 진행중
590 개 진행중
600 개 진행중
610 개 진행중
620 개 진행중
630 개 진행중
640 개 진행중
650 개 진행중
660 개 진행중
670 개 진행중
680 개 진행중
690 개 진행중
700 개 진행중
710 개 진행중
720 개 진행중
730 개 진행중
740 개 진행중
750 개 진행중
760 개 진행중
770 개 진행중
780 개 진행중
790 개 진행중
800 개 진행중
810 개 진행중
820 개 진행중
830 개 진행중
840 개 진행중
850 개 진행중
860 개 진행중
870 개 진행중
880 개 진행중
890 개 진행중
900 개 진행중
910 개 진행중
920 개 진행중
930 개 진행중
940 개 진행중
950 개 진행중
960 개 진행중


In [ ]:
# summary_list

In [ ]:
next_data['summary'] = summary_list

In [ ]:
# >>>>>>>>>>>>>>>>>> 훈련 반복할 때마다 파일명 바꾸기 default -> 1 -> 2...
next_data.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/kobart-kogpt_output/kobart_output_2.csv')

#### 5. 추가 train된 kobart에 tran set의 일기 데이터셋을 넣어 공감코멘트 생성 (.tsv)

In [ ]:
# train 파일 재지정
# kobart_output_n.csv 파일 정제 후 수행
import pandas as pd

# >>>>>>>>>>>>>>>>>> 훈련 반복할 때마다 파일명 바꾸기 default -> 1 -> 2...
temp= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/kobart-kogpt_output/kobart_output_2.csv', encoding='cp949')
temp= temp[['news', 'summary']]
temp.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBART-summarization/data/train.tsv', sep='\t')

#### 6. (4)로 돌아가 (4) ~ (5) 과정을 반복 (생성되는 공감코멘트가 어느정도의 성능으로 수렴할 때까지)